<a href="https://colab.research.google.com/github/amber3536/Wildfire_Prediction/blob/main/Resunet2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
"""Definition of ResUNet architecture"""
# Taken from https://github.com/nikhilroxtomar/Deep-Residual-Unet/blob/master/Deep%20Residual%20UNet.ipynb

from tensorflow import keras

def bn_act(x, act=True):
    x = keras.layers.BatchNormalization()(x)
    if act == True:
        x = keras.layers.Activation("relu")(x)
    return x

def conv_block(x, filters, kernel_size=(3, 3), padding="same", strides=1):
    conv = bn_act(x)
    conv = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides)(conv)
    return conv

def stem(x, filters, kernel_size=(3, 3), padding="same", strides=1):
    conv = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides)(x)
    conv = conv_block(conv, filters, kernel_size=kernel_size, padding=padding, strides=strides)

    shortcut = keras.layers.Conv2D(filters, kernel_size=(1, 1), padding=padding, strides=strides)(x)
    shortcut = bn_act(shortcut, act=False)

    output = keras.layers.Add()([conv, shortcut])
    return output

def residual_block(x, filters, kernel_size=(3, 3), padding="same", strides=1):
    res = conv_block(x, filters, kernel_size=kernel_size, padding=padding, strides=strides)
    res = conv_block(res, filters, kernel_size=kernel_size, padding=padding, strides=1)

    shortcut = keras.layers.Conv2D(filters, kernel_size=(1, 1), padding=padding, strides=strides)(x)
    shortcut = bn_act(shortcut, act=False)

    output = keras.layers.Add()([shortcut, res])
    return output

def upsample_concat_block(x, xskip):
    u = keras.layers.UpSampling2D((2, 2))(x)
    c = keras.layers.Concatenate()([u, xskip])
    return c

def get_model(input_shape):
    f = [16, 32, 64, 128, 256]
    inputs = keras.layers.Input((input_shape[0], input_shape[1], input_shape[2]))

    ## Encoder
    e0 = inputs
    e1 = stem(e0, f[0])
    e2 = residual_block(e1, f[1], strides=2)
    e3 = residual_block(e2, f[2], strides=2)
    e4 = residual_block(e3, f[3], strides=2)
    e5 = residual_block(e4, f[4], strides=2)

    ## Bridge
    b0 = conv_block(e5, f[4], strides=1)
    b1 = conv_block(b0, f[4], strides=1)

    ## Decoder
    u1 = upsample_concat_block(b1, e4)
    d1 = residual_block(u1, f[4])

    u2 = upsample_concat_block(d1, e3)
    d2 = residual_block(u2, f[3])

    u3 = upsample_concat_block(d2, e2)
    d3 = residual_block(u3, f[2])

    u4 = upsample_concat_block(d3, e1)
    d4 = residual_block(u4, f[1])

    outputs = keras.layers.Conv2D(1, (1, 1), padding="same", activation="sigmoid")(d4)
    model = keras.models.Model(inputs, outputs)
    return model

from keras import backend as K

# Focal Tversky_loss
def class_tversky(y_true, y_pred):
    smooth = 1

    y_true = K.permute_dimensions(y_true, (3,1,2,0))
    y_pred = K.permute_dimensions(y_pred, (3,1,2,0))

    y_true_pos = K.batch_flatten(y_true)
    y_pred_pos = K.batch_flatten(y_pred)
    true_pos = K.sum(y_true_pos * y_pred_pos, 1)
    false_neg = K.sum(y_true_pos * (1-y_pred_pos), 1)
    false_pos = K.sum((1-y_true_pos)*y_pred_pos, 1)
    alpha = 0.7
    return (true_pos + smooth)/(true_pos + alpha*false_neg + (1-alpha)*false_pos + smooth)

def focal_tversky_loss(y_true,y_pred):
    pt_1 = class_tversky(y_true, y_pred)
    gamma = 0.75
    return K.sum(K.pow((1-pt_1), gamma))

# Dice Loss
smooth = 1.
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def dice_coef_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)

#Keras
ALPHA = 0.8
GAMMA = 2

def focal_loss(targets, inputs, alpha=ALPHA, gamma=GAMMA):

    inputs = K.flatten(inputs)
    targets = K.flatten(targets)

    BCE = K.binary_crossentropy(targets, inputs)
    BCE_EXP = K.exp(-BCE)
    focal_loss = K.mean(alpha * K.pow((1-BCE_EXP), gamma) * BCE)

    return focal_loss

def dice_coef_binary(y_true, y_pred, smooth=1e-7):
    '''
    Dice coefficient for 2 categories. Ignores background pixel label 0
    Pass to model as metric during compile statement
    '''
    y_true_f = K.flatten(K.one_hot(K.cast(y_true, 'int32'), num_classes=2)[...,1:])
    y_pred_f = K.flatten(y_pred[...,1:])
    intersect = K.sum(y_true_f * y_pred_f, axis=-1)
    denom = K.sum(y_true_f + y_pred_f, axis=-1)
    return K.mean((2. * intersect / (denom + smooth)))


def dice_coef_binary_loss(y_true, y_pred):
    '''
    Dice loss to minimize. Pass to model as loss during compile statement
    '''
    return 1 - dice_coef_binary(y_true, y_pred)

def get_loss_function(loss_function_name):
    if loss_function_name == "focal_tversky_loss":
        loss_function = focal_tversky_loss
    elif loss_function_name == "dice_coef_loss":
        loss_function = dice_coef_loss
    elif loss_function_name == "dice_coef_binary_loss":
        loss_function = dice_coef_binary_loss
    elif loss_function_name == "focal_loss":
        loss_function = focal_loss
    elif loss_function_name == "sparse_categorical_crossentropy":
        loss_function = tf.keras.losses.SparseCategoricalCrossentropy()
    else:
        loss_function = loss_function_name # for keras implemented losses like "categorical_crossentropy"

    return loss_function

In [9]:
INPUT_FEATURES = ['elevation', 'wind_speed', 'wind_dir', 'tmin', 'tmax',
                  'landcover', 'precip', 'pdsi','solar', 'PrevFireMask']

In [10]:
import re
from typing import Dict, List, Optional, Text, Tuple
import matplotlib.pyplot as plt
from matplotlib import colors

import tensorflow as tf
# import model_satunet
import glob
import os
import sys


# Get loss function
loss_function = get_loss_function('dice_coef_loss')

# Define model architecture
model = get_model([64,64,10])

# Callbacks
!pip install wandb
import wandb
from wandb.keras import WandbCallback

callbacks = list()

# Optional: WandB callback config and init
config = {
    "dataset_id": "NDFP_data",
    "img_size": [64,64],
    "model_architecture": 'resunet',
    "num_layers_satunet": 4,
    "unfreeze_all_layers": False,
    "parent_model_name": None,
    "optimizer": 'adam',
    "learning_rate": 0.0001,
    "loss_function": 'dice_coef_loss',
    "epochs": 100,
    "batch_size": 100,
    "custom_objects": [
        "dice_coef",
        "focal_tversky_loss"
        ],
    "input_features": INPUT_FEATURES
    }
wandb.init(project='fire-model', config=config)
run_name = wandb.run.name
callbacks.append(WandbCallback())

# Define learning rate schedule callback
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    0.0001, decay_steps=15, decay_rate=0.96, staircase=True
    )

#Define checkpoints callback
checkpoint_path = os.path.join('./output', 'model', 'fire_model_{}.h5'.format(run_name))
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, save_weights_only=True, save_best_only=True
    )
callbacks.append(checkpoint_cb)

# Define Optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


In [11]:
# def get_dataset(file_pattern: Text, data_size: int, sample_size: int,
#                 batch_size: int, num_in_channels: int, compression_type: Text,
#                 clip_and_normalize: bool, clip_and_rescale: bool,
#                 random_crop: bool, center_crop: bool) -> tf.data.Dataset:
#     """Gets the dataset from the file pattern.

#     Args:
#     file_pattern: Input file pattern.
#     data_size: Size of tiles (square) as read from input files.
#     sample_size: Size the tiles (square) when input into the model.
#     batch_size: Batch size.
#     num_in_channels: Number of input channels.
#     compression_type: Type of compression used for the input files.
#     clip_and_normalize: True if the data should be clipped and normalized, False
#       otherwise.
#     clip_and_rescale: True if the data should be clipped and rescaled, False
#       otherwise.
#     random_crop: True if the data should be randomly cropped.
#     center_crop: True if the data shoulde be cropped in the center.

#     Returns:
#     A TensorFlow dataset loaded from the input file pattern, with features
#     described in the constants, and with the shapes determined from the input
#     parameters to this function.
#     """
#     if (clip_and_normalize and clip_and_rescale):
#         raise ValueError('Cannot have both normalize and rescale.')
#     dataset = tf.data.Dataset.list_files(file_pattern)
#     dataset = dataset.interleave(
#       lambda x: tf.data.TFRecordDataset(x, compression_type=compression_type),
#       num_parallel_calls=tf.data.experimental.AUTOTUNE)
#     dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
#     dataset = dataset.map(
#       lambda x: _parse_fn(  # pylint: disable=g-long-lambda
#           x, data_size, sample_size, num_in_channels, clip_and_normalize,
#           clip_and_rescale, random_crop, center_crop),
#       num_parallel_calls=tf.data.experimental.AUTOTUNE)
#     dataset = dataset.batch(batch_size)
#     dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
#     return dataset

In [12]:
# train_dataset = get_dataset(
#       file_pattern,
#       data_size=64,
#       sample_size=32,
#       batch_size=100,
#       num_in_channels=12,
#       compression_type=None,
#       clip_and_normalize=False,
#       clip_and_rescale=False,
#       random_crop=True,
#       center_crop=False)

# eval_dataset = get_dataset(
#       file_pattern_eval,
#       data_size=64,
#       sample_size=32,
#       batch_size=100,
#       num_in_channels=12,
#       compression_type=None,
#       clip_and_normalize=False,
#       clip_and_rescale=False,
#       random_crop=True,
#       center_crop=False)

In [13]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [14]:
man_length = 21550

MinMax = {'landcover':(10.0, 100.0),
              'tmax': (-41.95404185202824, 35.28747487720835),
              'tmin': (-42.4077221254351, 34.710974191122716),
              'wind_speed': (0.0002993076576944125, 14.27423496286687),
              'elevation': (-77.85292, 4379.4683),
              'wind_direction': (-179.99999083334689, 179.99999810000588),
              'solar_radiation': (-24.598765964771623, 964662.1183104622),
              'air_pressure': (99064.73425290409, 105551.80433948596),
              'precipitation': (-9.494998975299606e-05, 0.012117456275976952)}

In [15]:
import csv
import gzip
import numpy as np

In [16]:
def normalize(arr, min_value, max_value):
  arr = np.array(arr)
  return (arr - min_value) / (max_value - min_value)

In [17]:
def get_data_frp(i):
  u = []
  #with open("drive/MyDrive/day_fire_data/today_frp_" + str(i) + ".csv", mode ='r')as file:
  with gzip.open("drive/MyDrive/file/day_fire_data/today_frp/today_frp_" + str(i) + ".gz", mode ='rt')as file:
    csvFile = csv.reader(file)
    for lines in csvFile:
      p = []
      for y in lines:
        p.append(float(y))
      u.append(p)
  return u

def get_tomorrow_data_frp(i):
  u = []
  #with open("drive/MyDrive/day_fire_data/today_frp_" + str(i) + ".csv", mode ='r')as file:
  with gzip.open("drive/MyDrive/file/day_fire_data/tomorrow_frp/tomorrow_frp_" + str(i) + ".gz", mode ='rt')as file:
    csvFile = csv.reader(file)
    for lines in csvFile:
      p = []
      for y in lines:
        p.append(float(y))
      u.append(p)
  return u

def get_data_fire(i):
  u = []
  #with open("drive/MyDrive/day_fire_data/today_frp_" + str(i) + ".csv", mode ='r')as file:
  with gzip.open("drive/MyDrive/file/day_fire_data/today_fire/today_fire_" + str(i) + ".gz", mode ='rt')as file:
    csvFile = csv.reader(file)
    for lines in csvFile:
      p = []
      for y in lines:
        p.append(float(y))
      u.append(p)
  return u

def get_tomorrow_data_fire(i):
  u = []
  #with open("drive/MyDrive/day_fire_data/today_frp_" + str(i) + ".csv", mode ='r')as file:
  with gzip.open("drive/MyDrive/file/day_fire_data/tomorrow_fire/tomorrow_fire_" + str(i) + ".gz", mode ='rt')as file:
    csvFile = csv.reader(file)
    for lines in csvFile:
      p = []
      for y in lines:
        p.append(float(y))
      u.append(p)
  return u

def get_data_landcover(i):
  u = []
  with gzip.open("drive/MyDrive/file/day_fire_data/landcover/today_landcover_" + str(i) + ".gz", mode ='rt')as file:
    csvFile = csv.reader(file)
    for lines in csvFile:
      p = []
      for y in lines:
        p.append(float(y))
      u.append(p)
  return u

def get_data_elevation(i):
  u = []
  with gzip.open("drive/MyDrive/file/day_fire_data/elevation/today_elevation_" + str(i) + ".gz", mode ='rt')as file:
    csvFile = csv.reader(file)
    for lines in csvFile:
      p = []
      for y in lines:
        p.append(float(y))
      u.append(p)
  return u

def get_wind_speed(i):
  u = []
  with gzip.open("drive/MyDrive/file/day_fire_data/today_wind_speed/today_wind_speed_" + str(i) + ".gz", mode ='rt')as file:
    csvFile = csv.reader(file)
    for lines in csvFile:
      p = []
      for y in lines:
        p.append(float(y))
      u.append(p)
  return u

def get_air_pressure(i):
  u = []
  with gzip.open("drive/MyDrive/file/day_fire_data/today_air_pressure/today_air_pressure_" + str(i) + ".gz", mode ='rt')as file:
    csvFile = csv.reader(file)
    for lines in csvFile:
      p = []
      for y in lines:
        p.append(float(y))
      u.append(p)
  return u

def get_wind_direction(i):
  u = []
  with gzip.open("drive/MyDrive/file/day_fire_data/today_wind_direction/today_wind_direction_" + str(i) + ".gz", mode ='rt')as file:
    csvFile = csv.reader(file)
    for lines in csvFile:
      p = []
      for y in lines:
        p.append(float(y))
      u.append(p)
  return u

def get_temp_max(i):
  u = []
  with gzip.open("drive/MyDrive/file/day_fire_data/today_tmax/today_tmax_" + str(i) + ".gz", mode ='rt')as file:
    csvFile = csv.reader(file)
    for lines in csvFile:
      p = []
      for y in lines:
        p.append(float(y))
      u.append(p)
  return u

def get_temp_min(i):
  u = []
  with gzip.open("drive/MyDrive/file/day_fire_data/today_tmin/today_tmin_" + str(i) + ".gz", mode ='rt')as file:
    csvFile = csv.reader(file)
    for lines in csvFile:
      p = []
      for y in lines:
        p.append(float(y))
      u.append(p)
  return u

def get_solar_radiation(i):
  u = []
  with gzip.open("drive/MyDrive/file/day_fire_data/solar_radiation/today_solar_radiation_" + str(i) + ".gz", mode ='rt')as file:
    csvFile = csv.reader(file)
    for lines in csvFile:
      p = []
      for y in lines:
        p.append(float(y))
      u.append(p)
  return u

def get_precipitation(i):
  u = []
  with gzip.open("drive/MyDrive/file/day_fire_data/precipitation/today_precipitation_" + str(i) + ".gz", mode ='rt')as file:
    csvFile = csv.reader(file)
    for lines in csvFile:
      p = []
      for y in lines:
        p.append(float(y))
      u.append(p)
  return u

In [18]:
def _float_feature(value):
    """Returns a float_list from a list of float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))

In [19]:
def serialize_example(feature1, feature2, feature3, feature4, feature5, feature6, feature7, feature8, feature9, feature10, label):
    """
    Creates a tf.train.Example message ready to be written to a file.
    """
    feature = {
        'feature1': _float_feature(np.array(feature1).flatten()),
        'feature2': _float_feature(np.array(feature2).flatten()),
        'feature3': _float_feature(np.array(feature3).flatten()),
        'feature4': _float_feature(np.array(feature4).flatten()),
        'feature5': _float_feature(np.array(feature5).flatten()),
        'feature6': _float_feature(np.array(feature6).flatten()),
        'feature7': _float_feature(np.array(feature7).flatten()),
        'feature8': _float_feature(np.array(feature8).flatten()),
        'feature9': _float_feature(np.array(feature9).flatten()),
        'feature10': _float_feature(np.array(feature10).flatten()),
        'label': _float_feature(np.array(label).flatten()),
    }

    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    return example_proto.SerializeToString()

In [20]:
# Number of images and labels in your dataset
num_examples = 21550
#num_examples = 20

# Output file name
tfrecord_file = 'drive/MyDrive/file/data.tfrecords'

# with tf.io.TFRecordWriter(tfrecord_file) as writer:
#     for idx in range(num_examples):
#         # Load your 64x64 feature image and label
#         #feature0, feature1, feature2, feature3,  label = load_data(i)  # Implement this function
#         feature1 = get_data_fire(idx)
#         feature2 = normalize(get_data_elevation(idx), MinMax["elevation"][0], MinMax["elevation"][1])
#         feature3 = normalize(get_wind_speed(idx), MinMax["wind_speed"][0], MinMax["wind_speed"][1])
#         feature4 = normalize(get_air_pressure(idx), MinMax["air_pressure"][0], MinMax["air_pressure"][1])
#         feature5 = normalize(get_wind_direction(idx), MinMax["wind_direction"][0], MinMax["wind_direction"][1])
#         feature6 = normalize(get_temp_max(idx), MinMax["tmax"][0], MinMax["tmax"][1])
#         feature7 = normalize(get_temp_min(idx), MinMax["tmin"][0], MinMax["tmin"][1])
#         feature8 = normalize(get_solar_radiation(idx), MinMax["solar_radiation"][0], MinMax["solar_radiation"][1])
#         feature9 = normalize(get_precipitation(idx), MinMax["precipitation"][0], MinMax["precipitation"][1])
#         feature10 = normalize(get_data_landcover(idx), MinMax["landcover"][0], MinMax["landcover"][1])
#         label = get_tomorrow_data_fire(idx)
#         # Serialize example and write it to the TFRecord file
#         example = serialize_example(feature1, feature2, feature3, feature4, feature5, feature6, feature7, feature8, feature9, feature10, label)
#         writer.write(example)


In [30]:
def _parse_function(proto):

    # keys_to_features = {
    #     'feature1': tf.io.VarLenFeature(tf.float32),
    #     'feature2': tf.io.VarLenFeature(tf.float32),
    #     'feature3': tf.io.VarLenFeature(tf.float32),
    #     'feature4': tf.io.VarLenFeature(tf.float32),
    #     'feature5': tf.io.VarLenFeature(tf.float32),
    #     'feature6': tf.io.VarLenFeature(tf.float32),
    #     'feature7': tf.io.VarLenFeature(tf.float32),
    #     'feature8': tf.io.VarLenFeature(tf.float32),
    #     'feature9': tf.io.VarLenFeature(tf.float32),
    #     'feature10': tf.io.VarLenFeature(tf.float32),
    #     'label': tf.io.VarLenFeature(tf.float32)
    # }
    keys_to_features = {
        'feature1': tf.io.FixedLenFeature([64*64], tf.float32),
        'feature2': tf.io.FixedLenFeature([64*64], tf.float32),
        'feature3': tf.io.FixedLenFeature([64*64], tf.float32),
        'feature4': tf.io.FixedLenFeature([64*64], tf.float32),
        'feature5': tf.io.FixedLenFeature([64*64], tf.float32),
        'feature6': tf.io.FixedLenFeature([64*64], tf.float32),
        'feature7': tf.io.FixedLenFeature([64*64], tf.float32),
        'feature8': tf.io.FixedLenFeature([64*64], tf.float32),
        'feature9': tf.io.FixedLenFeature([64*64], tf.float32),
        'feature10': tf.io.FixedLenFeature([64*64], tf.float32),
        'label': tf.io.FixedLenFeature([64*64], tf.float32),
    }

    # Load one example
    parsed_features = tf.io.parse_single_example(proto, keys_to_features)

    features = [tf.reshape(parsed_features[f'feature{i}'], [64, 64]) for i in range(10)]
    label = tf.reshape(parsed_features['label'], [64, 64])

    # Stack features along a new axis if your model expects a single input tensor
    stacked_features = tf.stack(features, axis=-1)
    # feature_list = []
    # for i in range(1, 11):
    #     feature = tf.reshape(tf.sparse.to_dense(parsed_features[f'feature{i}']), [64, 64])
    #     feature_list.append(feature)

    # combined_features = tf.stack(feature_list, axis=-1)
    # label = tf.reshape(tf.sparse.to_dense(parsed_features['label']), [64, 64])


    return stacked_features, label


In [22]:

# def data_generator():
#     for idx in range(man_length):  # Replace 'n' with the total number of file
#       feature1 = get_data_fire(idx)
#       feature2 = normalize(get_data_elevation(idx), MinMax["elevation"][0], MinMax["elevation"][1])
#       feature3 = normalize(get_wind_speed(idx), MinMax["wind_speed"][0], MinMax["wind_speed"][1])
#       feature4 = normalize(get_air_pressure(idx), MinMax["air_pressure"][0], MinMax["air_pressure"][1])
#       feature5 = normalize(get_wind_direction(idx), MinMax["wind_direction"][0], MinMax["wind_direction"][1])
#       feature6 = normalize(get_temp_max(idx), MinMax["tmax"][0], MinMax["tmax"][1])
#       feature7 = normalize(get_temp_min(idx), MinMax["tmin"][0], MinMax["tmin"][1])
#       feature8 = normalize(get_solar_radiation(idx), MinMax["solar_radiation"][0], MinMax["solar_radiation"][1])
#       feature9 = normalize(get_precipitation(idx), MinMax["precipitation"][0], MinMax["precipitation"][1])
#       feature10 = normalize(get_data_landcover(idx), MinMax["landcover"][0], MinMax["landcover"][1])

#       label = get_tomorrow_data_fire(idx)
#       combined_features = np.stack([feature1, feature2, feature3, feature4, feature5, feature6, feature7, feature8, feature9, feature10], axis=-1)
#       yield (combined_features, label)

In [23]:
def count_records(tfrecord_file):
    count = 0
    for _ in tf.data.TFRecordDataset(tfrecord_file):
        count += 1
    return count

In [24]:
#count_records(tfrecord_file)

In [25]:
# dataset = tf.data.Dataset.from_generator(
#     data_generator,
#     output_types=(tf.float32, tf.float32),
#     output_shapes=((64, 64, 10), (64, 64))
# )

tfrecord_file = 'drive/MyDrive/file/data.tfrecords'

dataset = tf.data.TFRecordDataset(tfrecord_file)

dataset = dataset.map(_parse_function)


In [26]:
dataset = dataset.shuffle(buffer_size=1000).batch(32).prefetch(tf.data.AUTOTUNE)

In [27]:
train_size = int(0.7 * man_length)
val_size = int(0.2 * man_length)

# Split the dataset
train_dataset = dataset.take(train_size)
remaining_dataset = dataset.skip(train_size)
eval_dataset = remaining_dataset.take(val_size)
test_dataset = remaining_dataset.skip(val_size)

In [28]:
# for features, label in train_dataset.take(1):
#     print("Features shape:", features.shape)  # Should match your model's input shape
#     print("Label shape:", label.shape)

In [29]:
# Compile and train model
model.compile(
    optimizer=optimizer,
    loss=loss_function, metrics=[dice_coef,
                                 tf.keras.metrics.AUC(curve="PR"),
                                 tf.keras.metrics.Precision(),
                                 tf.keras.metrics.Recall()
                                ]
    )

#model.fit(train_dataset, epochs=5)
history = model.fit(
    train_dataset,
    validation_data=eval_dataset,
    epochs=100,
    callbacks=callbacks
)

Epoch 1/100
    240/Unknown - 1209s 5s/step - loss: 0.8297 - dice_coef: 0.1703 - auc: 0.0579 - precision: 0.0699 - recall: 0.4237

InvalidArgumentError: ignored